In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


In [ ]:

'''
这个代码块使用了1994年到2020年(预测周期之前)的所有数据训练了一个BILSTM模型，作为初始模型
在查询相关资料后，我们使用RSI和开盘价作为特征(来预测收盘价),损失函数使用MSE
为了方便使用,我们已经提供了在开发时训练好的模型new_model_RSI.h5
'''

# 创建数据集的函数
def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :-1])  # 包含 time_step 天的所有特征
        Y.append(data[i + time_step, -1])       # 预测下一个时间点的 'Close' 价
    return np.array(X), np.array(Y)

# 加载数据
hsi_data = pd.read_csv('HSI_Date_with_RSI.csv')
hsi_data['Date'] = pd.to_datetime(hsi_data['Date'])
data = hsi_data[['Open', 'RSI', 'Close']].values  # 选取开盘价、RSI和收盘价

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# 划分数据
time_step = 60
X_train, y_train = create_dataset(data_scaled, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 2)  # 特征：Open和RSI

# 使用BiLSTM模型
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(time_step, 2)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer=Adam(0.01))

# 训练模型
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

# 保存模型
model.save('new_model_RSI.h5')


In [ ]:

'''

基于上一个代码块训练得出的初始模型我们对投资周期内的收盘价进行滚动预测
具体步骤如下：
1.用初始模型对下一个月的每一天逐日预测
2.在预测完这一个月后，将这一个月的真实收盘价添加进原来的训练集(先预测后添加,避免信息泄露)
3.再次训练模型(也可以微调)
4.用训练出的新模型再向后预测一个月
5.重复以上步骤,直到投资周期结束
6.可视化预测结果

'''

import matplotlib.pyplot as plt
from keras.models import load_model


# 加载模型
model = load_model('new_model_RSI.h5')

# 预测并更新模型的玩意
def predict_and_update(data, start_date, end_date, model, scaler, time_step=60):
    current_date = start_date
    predicted_closes = []
    prediction_dates = []

    while current_date <= end_date:
        # 提取最近 time_step 天的数据进行预测（炼丹开始）
        recent_data = data[data['Date'] < current_date][-time_step:]
        recent_scaled = scaler.transform(recent_data[['Open', 'RSI', 'Close']])
        X_test = recent_scaled[:, :-1].reshape(1, time_step, 2)  # 使用 'Open' 和 'RSI' 作为特征
        predicted_close = model.predict(X_test)[0][0]
        predicted_closes.append(predicted_close)
        prediction_dates.append(current_date)

        # 下一天的预测
        current_date += pd.DateOffset(days=1)

    return prediction_dates, predicted_closes


# 滚动预测和更新
start_date = pd.Timestamp('2020-11-17')
end_date = pd.Timestamp('2023-11-17')
current_date = start_date
all_prediction_dates = []
all_actual_prices = []
all_predicted_prices = []

while current_date <= end_date:
    next_month_end = current_date + pd.DateOffset(months=1) - pd.DateOffset(days=1)

    # 预测下一个月的价格
    prediction_dates, predicted_prices = predict_and_update(hsi_data, current_date, next_month_end, model, scaler)
    
    # 更新训练集
    new_data = hsi_data[(hsi_data['Date'] >= current_date) & (hsi_data['Date'] <= next_month_end)]
    new_scaled = scaler.fit_transform(new_data[['Open', 'RSI', 'Close']])
    data_scaled = np.append(data_scaled, new_scaled, axis=0)

    # 准备微调模型的数据
    X_train, y_train = create_dataset(data_scaled, time_step)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 2)  # 2个特征：Open和RSI

    # 微调模型
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

    # 时移一个月
    current_date = next_month_end + pd.DateOffset(days=1)

    # 反归一化预测价格
    predicted_full = np.zeros((len(predicted_prices), 3))
    predicted_full[:, -1] = predicted_prices  # 预测的收盘价
    predicted_prices_scaled = scaler.inverse_transform(predicted_full)[:, -1]

    # 保存预测价格和日期
    all_prediction_dates.extend(prediction_dates)
    all_predicted_prices.extend(predicted_prices_scaled)


actual_dates = hsi_data[(hsi_data['Date'] >= start_date) & (hsi_data['Date'] <= end_date)]['Date']
actual_prices = hsi_data[(hsi_data['Date'] >= start_date) & (hsi_data['Date'] <= end_date)]['Close']

'''

预测数据存在all_prediction_dates, all_predicted_prices的两个list里面，可以拿出来做量化交易。
我已经用pd.to_csv()在路径下中保存了一份predicted_prices的csv数据文件,可以直接使用

'''

# 可视化
plt.figure(figsize=(15, 7))
plt.plot(all_prediction_dates, all_predicted_prices, label='Predicted Prices', color='red')
plt.plot(actual_dates, actual_prices, label='Actual Prices', color='blue')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
